# Task 1

---

## Web scraping and analysis

This Jupyter notebook includes some code to get you started with web scraping. We will use a package called `BeautifulSoup` to collect the data from the web. Once you've collected your data and saved it into a local `.csv` file you should start with your analysis.

### Scraping data from Skytrax

If you visit [https://www.airlinequality.com] you can see that there is a lot of data there. For this task, we are only interested in reviews related to British Airways and the Airline itself.

If you navigate to this link: [https://www.airlinequality.com/airline-reviews/british-airways] you will see this data. Now, we can use `Python` and `BeautifulSoup` to collect all the links to the reviews and then to collect the text data on each of the individual review links.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 100
page_size = 100

reviews = []

# for i in range(1, pages + 1):
for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())
    
    print(f"   ---> {len(reviews)} total reviews")

In [ ]:
df = pd.DataFrame()
df["reviews"] = reviews
df.head()

In [ ]:
df.to_csv("BA_reviews.csv")

Congratulations! Now you have your dataset for this task! The loops above collected 1000 reviews by iterating through the paginated pages on the website. However, if you want to collect more data, try increasing the number of pages!

 The next thing that you should do is clean this data to remove any unnecessary text from each of the rows. For example, "✅ Trip Verified" can be removed from each row if it exists, as it's not relevant to what we want to investigate.

In [ ]:
df

In [ ]:
df['reviews']=df["reviews"].str.replace('✅ Trip Verified |','')
df['reviews']=df["reviews"].str.replace('|','')
df['reviews']=df["reviews"].str.strip()

In [ ]:
df

In [ ]:
df = df[~df['reviews'].str.contains('Not Verified')]

# Resetting the index after filtering
df.reset_index(drop=True, inplace=True)

In [ ]:
df

NOW WE DO SENTIMENT AND CONTEXTUAL ANALYASIS ON THE FOLLWING DATA

In [ ]:
# Assuming df is your DataFrame
# Resetting the index with continuous numbers starting from 1
df.reset_index(drop=True, inplace=True)

# Display the DataFrame after resetting the index
print(df)
df = df.reset_index(drop=True)



In [ ]:
import re

def remove_emojis(text):
    if isinstance(text, str):
        emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"
                               u"\U0001F300-\U0001F5FF"
                               u"\U0001F680-\U0001F6FF"
                               u"\U0001F1E0-\U0001F1FF"
                               u"\U00002500-\U00002BEF"
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"
                               u"\u3030"
                               "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', text)
    else:
        return text

df['reviews'] = df['reviews'].apply(remove_emojis)


In [ ]:
df

In [ ]:
import string

# Function to remove punctuation
def remove_punctuation(text):
    if isinstance(text, str):
        return text.translate(str.maketrans('', '', string.punctuation))
    else:
        return text
df['reviews']=df['reviews'].apply(remove_punctuation)

In [ ]:
from spellchecker import SpellChecker

# Function to perform spell checking
def correct_spelling(text):
    if isinstance(text, str):
        spell = SpellChecker()
        words = text.split()
        corrected_words = [spell.correction(word) if word.isalpha() else word for word in words]
        cleaned_words = [word for word in corrected_words if word is not None and isinstance(word, str)]
        return ' '.join(cleaned_words)
    else:
        return text

# Take a smaller sample of the data (e.g., the first 1000 rows)
sample_size = 1000
sample = df['reviews'].iloc[:sample_size]

# Apply the function to the sample
df['reviews'].iloc[:sample_size] = sample.apply(correct_spelling)

In [ ]:
df.head()

In [ ]:
df['reviews'] = df['reviews'].apply(lambda x: x.lower() if isinstance(x, str) else x)

In [ ]:
df.tail(5)

TOKENIZATON

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

# Download NLTK resources (if you haven't already)
nltk.download('punkt')

# Tokenize the 'reviews' column
df['reviews_tokens'] = df['reviews'].apply(lambda x: word_tokenize(x) if isinstance(x, str) else x)


In [ ]:
df.head()

STOPWORD REMOVAL

In [ ]:
from nltk.corpus import stopwords

# Download NLTK stopwords (if you haven't already)
nltk.download('stopwords')

# Get the English stopwords
stop_words = set(stopwords.words('english'))

# Remove 'not' from the stop words set
stop_words.discard('not')

# Function to remove stopwords
def remove_stopwords(tokens):
    if isinstance(tokens, list):
        filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
        return filtered_tokens
    else:
        return tokens

# Apply stopword removal to the tokenized column
df['reviews_tokens_without_stopwords'] = df['reviews_tokens'].apply(remove_stopwords)


In [ ]:
df.head()

In [ ]:
from nltk.stem import PorterStemmer

# Initialize the Porter Stemmer
porter = PorterStemmer()

# Function to perform stemming
def perform_stemming(tokens):
    if isinstance(tokens, list):
        stemmed_tokens = [porter.stem(word) for word in tokens]
        return stemmed_tokens
    else:
        return tokens

# Apply stemming to the tokens without stopwords
df['reviews_stemmed'] = df['reviews_tokens_without_stopwords'].apply(perform_stemming)

In [ ]:
from nltk.stem import WordNetLemmatizer

# Initialize the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Function to perform lemmatization
def perform_lemmatization(tokens):
    if isinstance(tokens, list):
        lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
        return lemmatized_tokens
    else:
        return tokens

# Apply lemmatization to the stemmed tokens
df['reviews_lemmatized'] = df['reviews_stemmed'].apply(perform_lemmatization)

In [ ]:
from textblob import TextBlob

# Function to find sentiment polarity and label
def find_sentiment(text):
    if isinstance(text, list):
        analysis = TextBlob(' '.join(text))
        polarity = analysis.sentiment.polarity

        # Classify sentiment based on polarity
        if polarity > 0:
            sentiment = 'Positive'
        elif polarity < 0:
            sentiment = 'Negative'
        else:
            sentiment = 'Neutral'

        return polarity, sentiment
    else:
        return None, None

# Apply sentiment analysis and labeling
df['sentiment_polarity'], df['sentiment'] = zip(*df['reviews_lemmatized'].apply(find_sentiment))


In [ ]:
df.head()

In [ ]:
df.to_csv('BAA.csv')

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'sentiment_polarity' is the column in your DataFrame containing sentiment polarity scores
plt.figure(figsize=(10, 6))
plt.hist(df['sentiment_polarity'], bins=50, color='skyblue', edgecolor='black')
plt.title('Distribution of Sentiment Polarity Scores')
plt.xlabel('Sentiment Polarity')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assuming 'sentiment' is the column in your DataFrame indicating sentiment
sentiment_counts = df['sentiment'].value_counts()

# Create a bar plot
plt.figure(figsize=(8, 6))
sentiment_counts.plot(kind='bar', color=['green', 'red', 'gray'])
plt.title('Distribution of Sentiments')
plt.xlabel('Sentiment')
plt.ylabel('Frequency')
plt.xticks(rotation=0)
plt.show()


In [ ]:
plt.figure(figsize=(8, 8))
plt.pie(sentiment_counts, labels=sentiment_counts.index, autopct='%1.1f%%', colors=['green', 'red', 'gray'])
plt.title('Distribution of Sentiments')
plt.show()


In [ ]:
# Assuming 'sentiment' is the column in your DataFrame indicating sentiment

# Create separate DataFrames for positive, negative, and neutral sentiments
positive_df = df[df['sentiment'] == 'Positive']
negative_df = df[df['sentiment'] == 'Negative']
neutral_df = df[df['sentiment'] == 'Neutral']

# Save each DataFrame to a separate CSV file
positive_df.to_csv('positive_reviews.csv', index=False)
negative_df.to_csv('negative_reviews.csv', index=False)
neutral_df.to_csv('neutral_reviews.csv', index=False)


In [ ]:
pip install scipy==1.7.3


In [ ]:
# Load the CSV file containing negative reviews
negative_df = pd.read_csv('negative_reviews.csv')

# Assuming 'reviews' is the column in your DataFrame containing the negative reviews
negative_reviews = negative_df['reviews']

# Find the top 5 repeated sentences
top_repeated_sentences = negative_reviews.value_counts().head(5)

# Display the top 5 repeated sentences
print("Top 5 Repeated Sentences in Negative Reviews:")
print(top_repeated_sentences)
